## Create a Vectordb from Amazon Product Dataset (pickle files)

In [7]:
import os
from tqdm import tqdm
from dotenv import load_dotenv
from huggingface_hub import login
import pickle
from sentence_transformers import SentenceTransformer
import chromadb

### Internal Classes

In [8]:
### Environment 

load_dotenv(override=True)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
DB = "../products_vectorstore"
PRODUCTION_DB = "../products_vectordb_production"

In [9]:
hf_token = os.environ["HF_TOKEN"]
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Import the (previously cleaned and formatted) amazon product dataset (as pkl files)

Resource: https://drive.google.com/drive/folders/1f_IZGybvs9o0J5sb3xmtTEQB3BXllzrW?usp=drive_link 

In [10]:
### Want to create cvector db for local development or production?
### If DEV_MODE = false(for production code), lighter chromadb will be created due to the hosting file size limit
DEV_MODE = False

In [11]:
with open("train.pkl", "rb") as f:
    train = pickle.load(f) if DEV_MODE else pickle.load(f)[:100000]

In [12]:
print(train[0].prompt)

How much does this cost to the nearest dollar?

Main_Category: Toys & Games

CoComelon Official Musical Tractor w/Sounds & Exclusive 3-inch Farm JJ Toy, Play a Clip of “Old Macdonald” Song Plus More Sounds and Phrases
E-I-E-I WOAH! Get ready for a melodious ride through the farm with the CoComelon Musical Tractor! This brightly colored tractor comes with an exclusive 3-inch figure of JJ, a popular character from the educational CoComelon YouTube series! This musical CoComelon toy can seat any of your 3-inch CoComelon figures. Press on the grill of the tractor and listen as a clip of the “Old MacDonald” song plays for you and your child to sing along to! This exciting musical childrens toy also plays animal and tractor sounds, and even the CoComelon intro

Price is $13.00


In [13]:
len(train)

100000

## ChromaDB for All Items (for local dev) 
## OR 
## Smaller ChromaDB for Production/Deployment

In [ ]:

DB_PATH = DB if DEV_MODE else PRODUCTION_DB
collection_name = "products"

client = chromadb.PersistentClient(path=DB_PATH)
collection = client.get_or_create_collection(collection_name)    

In [ ]:
### Use the existing collection, otherwise create it if none of it.
###### NOTE: simply get_or_create_collection does the job perfectly without this messy lines of code

# collection_name = "products"

# existing_collection_names = client.list_collections()

# if any(c.name == collection_name for c in existing_collection_names): 
#     print("Collection already exists. Using it.")
#     collection = client.get_collection(collection_name)
# else: 
#     print("Creating new collection.")
#     collection = client.create_collection(collection_name)

## SentenceTransfomer

In [ ]:
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
### Test:

vector = embedding_model.encode("hi there!")

vector

In [ ]:
### Quick sidebar - Good mental model of how SSentenceTransformer works

import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%")

In [ ]:
how_similar("Java", "C++")
how_similar("Java", "mug")
how_similar("Cup of Java", "mug")

In [ ]:
def description(item): 
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

## Populate RAG Datastore 
With 150,000 items in Chroma.

In [ ]:
NUMBER_OF_DOCS = len(train)

for i in tqdm(range(0, NUMBER_OF_DOCS, 1000)):
    documents = [description(item) for item in train[i:i+1000]]
    vectors = embedding_model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]

    collection.add(
        ids=ids, 
        documents=documents, 
        embeddings=vectors, 
        metadatas=metadatas
    )